In [5]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import svm, metrics
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
from sklearn.ensemble import RandomForestClassifier
# from google.colab import files
import re
from collections import OrderedDict
import warnings
warnings.filterwarnings('ignore')

In [2]:
uploaded = files.upload()

Saving test.csv to test (2).csv
Saving train.csv to train (2).csv


In [6]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [7]:
survived = train['Survived']
train.drop(labels=['Survived'], axis=1, inplace = True)
train.insert(11, 'Survived', survived)
# train.drop(labels=['Name','Ticket','Cabin'], axis=1, inplace=True)

In [4]:
# train.groupby('Survived').mean()

In [5]:
# pd.crosstab(train['Pclass'],train['Survived']).plot(kind='bar')

In [6]:
# pd.crosstab(train['Sex'],train['Survived']).plot(kind='bar')

In [7]:
# train['Age'].plot.hist(bins=25)

In [8]:
# train_copy = train

In [9]:
# for i in range(len(train_copy)):
#     if train_copy['Age'].iloc[i]>0 and train_copy['Age'].iloc[i]<=10:
#         train_copy['Age'].iloc[i] = 0
#     elif train_copy['Age'].iloc[i]>10 and train_copy['Age'].iloc[i]<=30:
#         train_copy['Age'].iloc[i] = 1
#     elif train_copy['Age'].iloc[i]>30 and train_copy['Age'].iloc[i]<=50:
#         train_copy['Age'].iloc[i] = 2
#     elif train_copy['Age'].iloc[i]>50 and train_copy['Age'].iloc[i]<=max(train['Age']):
#         train_copy['Age'].iloc[i] = 3

# pd.crosstab(train_copy['Age'],train_copy['Survived']).plot(kind='bar')

In [10]:
# pd.crosstab(train['SibSp'],train['Survived']).plot(kind='bar')

In [11]:
# pd.crosstab(train['Parch'],train['Survived']).plot(kind='bar')

In [12]:
# pd.crosstab(train['Embarked'],train['Survived']).plot(kind='bar')

In [13]:
# quart = np.quantile(train['Fare'], [0.5,0.75,1])
# for i in range(len(train_copy)):
#     if train_copy['Fare'].iloc[i]<=quart[0]:
#         train_copy['Fare'].iloc[i] = 0
#     elif train_copy['Fare'].iloc[i]>quart[0] and train_copy['Fare'].iloc[i]<=quart[1]:
#         train_copy['Fare'].iloc[i] = 1
#     elif train_copy['Fare'].iloc[i]>quart[1] and train_copy['Fare'].iloc[i]<=quart[2]:
#         train_copy['Fare'].iloc[i] = 2

# pd.crosstab(train_copy['Fare'],train_copy['Survived']).plot(kind='bar')

In [8]:
sex_idx = {'male': 0, 'female': 1}
embarked_idx = {'S':1, 'C':2, 'Q':3}
for i in range(len(train)):
    train['Sex'].iloc[i] = sex_idx[train['Sex'].iloc[i]]
    if train['Embarked'].iloc[i]!=train['Embarked'].iloc[i]:
        continue
    else:
        train['Embarked'].iloc[i] = embarked_idx[train['Embarked'].iloc[i]]

In [15]:
# pclass1 = train[train['Pclass']==1]
# for i in range(len(pclass1)):
#     if pclass1['Cabin'].iloc[i]==pclass1['Cabin'].iloc[i]:
#         pclass1['Cabin'].iloc[i] = ''.join((x for x in pclass1['Cabin'].iloc[i] if not x.isdigit()))
#         p=""
#         for char in pclass1['Cabin'].iloc[i]:
#             if char not in p:
#                 p=p+char
#         pclass1['Cabin'].iloc[i]=p

# pclass1['Cabin'] = pclass1['Cabin'].str.strip()
# # pclass1.hist(column='Cabin')
# pclass1['Cabin'].value_counts().plot(kind='bar')

In [16]:
# train['Age'] = train['Age'].fillna(train['Age'].mean())

In [17]:
# train.isna().sum()

In [9]:
train.dropna(axis=0, subset=['Embarked'], inplace=True)

In [10]:
for i in range(len(test)):
    test['Sex'].iloc[i] = sex_idx[test['Sex'].iloc[i]]
    if test['Embarked'].iloc[i]!=test['Embarked'].iloc[i]:
        continue
    else:
        test['Embarked'].iloc[i] = embarked_idx[test['Embarked'].iloc[i]]

In [20]:
# test.isna().sum()

In [11]:
# x_train = train[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
# y_train = train['Survived']
# x_test = test[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
x = train[['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked']]
y = train['Survived']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

In [12]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
# x_test_scaled = scaler.transform(x_test)
x_train_scaled

array([[ 0.83058357, -0.75238241, -0.48220821, -0.49921801, -0.4814764 ,
        -0.56034932],
       [-1.54585474, -0.75238241, -0.48220821, -0.49921801, -0.10404863,
         1.04613603],
       [-1.54585474, -0.75238241, -0.48220821, -0.49921801, -0.12651301,
        -0.56034932],
       ...,
       [-1.54585474, -0.75238241, -0.48220821, -0.49921801, -0.63593344,
        -0.56034932],
       [-1.54585474,  1.32911134, -0.48220821, -0.49921801, -0.08502067,
         1.04613603],
       [-0.35763558, -0.75238241,  0.3813939 ,  0.72052085,  0.06919653,
        -0.56034932]])

In [13]:
C = [round(x,2) for x in np.linspace(0.1,3,5)]
kernel = ['linear', 'poly', 'rbf']
degree = list(range(5,9))
gamma = ['scale', 'auto']
coef0 = [round(x,2) for x in np.linspace(-5,5,5)]
class_weight = [None, 'balanced']
# random_state = []

random_grid = {'C': C, 'kernel': kernel, 'degree': degree, 'gamma': gamma, 'coef0': coef0, 'class_weight': class_weight}
random_grid

{'C': [0.1, 0.82, 1.55, 2.28, 3.0],
 'kernel': ['linear', 'poly', 'rbf'],
 'degree': [5, 6, 7, 8],
 'gamma': ['scale', 'auto'],
 'coef0': [-5.0, -2.5, 0.0, 2.5, 5.0],
 'class_weight': [None, 'balanced']}

In [ ]:
model_test = svm.SVC()
svm_random = RandomizedSearchCV(estimator=model_test, param_distributions=random_grid, n_iter=500, cv=4, verbose=2)
svm_random.fit(x_train_scaled,y_train)

print(svm_random.best_params_)

Fitting 4 folds for each of 500 candidates, totalling 2000 fits
[CV] END C=0.1, class_weight=None, coef0=5.0, degree=7, gamma=scale, kernel=poly; total time=   0.1s
[CV] END C=0.1, class_weight=None, coef0=5.0, degree=7, gamma=scale, kernel=poly; total time=   0.0s
[CV] END C=0.1, class_weight=None, coef0=5.0, degree=7, gamma=scale, kernel=poly; total time=   0.2s
[CV] END C=0.1, class_weight=None, coef0=5.0, degree=7, gamma=scale, kernel=poly; total time=   0.2s
[CV] END C=0.1, class_weight=None, coef0=-2.5, degree=5, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END C=0.1, class_weight=None, coef0=-2.5, degree=5, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END C=0.1, class_weight=None, coef0=-2.5, degree=5, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END C=0.1, class_weight=None, coef0=-2.5, degree=5, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END C=3.0, class_weight=balanced, coef0=0.0, degree=8, gamma=scale, kernel=poly; total time=   0.0s
[CV] END C=3.0, class_w

In [ ]:
clfr = svm.SVC(kernel= 'rbf',
 gamma= 'scale',
 degree= 5,
 coef0= -2.5,
 class_weight= None,
 C= 0.1)
clfr.fit(x_train_scaled,y_train)

SVC(C=0.1, coef0=-2.5, degree=5)

In [ ]:
y_pred = clfr.predict(x_test)
print("Accuracy on test set",metrics.accuracy_score(y_test, y_pred))


Accuracy on test set 0.6643356643356644


In [ ]:
x_train_scaled = x_train.astype('float32')
x_test = x_test.astype('float32')

In [ ]:
nn_seq = Sequential()
nn_seq.add(Dense(12, input_shape=(7,), activation='relu'))
nn_seq.add(Dense(8, activation='relu'))
nn_seq.add(Dense(1, activation='sigmoid'))
print(nn_seq)

In [ ]:
model = Sequential()
model.add(Dense(12, input_shape=(7,), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(x_train_scaled, y_train, epochs=150, batch_size=3)
# evaluate the keras model
_, accuracy = model.evaluate(x_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))
_i, accuracy1 = model.evaluate(x_train_scaled, y_train)
print('Accuracy on training: %.2f' % (accuracy1*100))

In [ ]:
predictions = (model.predict(x_train) > 0.5).astype(int)
for i in range(len(x_train)):
	  print(predictions[i], y_train.iloc[i])

In [ ]:
n_estimators = [int(x) for x in np.linspace(0,2000,10)]
max_features = ['sqrt', 'log2', None]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
class_weight = ['balanced', 'balancede_subsample', None]

random_grid_rf = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'class_weight': class_weight}
random_grid_rf

{'n_estimators': [0, 222, 444, 666, 888, 1111, 1333, 1555, 1777, 2000],
 'max_features': ['sqrt', 'log2', None],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110],
 'min_samples_split': [2, 5, 10],
 'min_samples_leaf': [1, 2, 4],
 'bootstrap': [True, False],
 'class_weight': ['balanced', 'balancede_subsample', None]}

In [ ]:
rf_cv = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator=rf_cv, param_distributions=random_grid_rf, n_iter=200, cv=3, verbose=2)
rf_random.fit(x_train_scaled, y_train)

Fitting 3 folds for each of 200 candidates, totalling 600 fits
[CV] END bootstrap=True, class_weight=balanced, max_depth=50, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=666; total time=   1.5s
[CV] END bootstrap=True, class_weight=balanced, max_depth=50, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=666; total time=   1.2s
[CV] END bootstrap=True, class_weight=balanced, max_depth=50, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=666; total time=   1.1s
[CV] END bootstrap=False, class_weight=balancede_subsample, max_depth=90, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=444; total time=   0.0s
[CV] END bootstrap=False, class_weight=balancede_subsample, max_depth=90, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=444; total time=   0.0s
[CV] END bootstrap=False, class_weight=balancede_subsample, max_depth=90, max_features=log2, min_samples_leaf=1, mi

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=200,
                   param_distributions={'bootstrap': [True, False],
                                        'class_weight': ['balanced',
                                                         'balancede_subsample',
                                                         None],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [0, 222, 444, 666, 888,
                                                         1111, 1333, 1555, 1777,
                                                         2000]},
                   verbose=2)

In [ ]:
rf_random.best_params_

{'n_estimators': 666,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': None,
 'max_depth': 80,
 'class_weight': None,
 'bootstrap': False}

In [ ]:
rf_model = RandomForestClassifier(n_estimators= 666,
 min_samples_split= 10,
 min_samples_leaf= 1,
 max_features= None,
 max_depth= 80,
 class_weight= None,
 bootstrap= False)

rf_model.fit(x_train_scaled,y_train)
y_pred = rf_model.predict(x_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6643356643356644


In [ ]:
C = [round(x,2) for x in np.linspace(0.1,5,10)]
class_weight = ['balanced', None]
fit_intercept = [True, False]
multi_class = ['auto', 'ovr', 'multinomial']
penalty = ['l2', None]
solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
warm_start = [True, False]

random_grid_logreg = {'C': C, 'class_weight': class_weight, 'fit_intercept':fit_intercept, 'multi_class':multi_class, 'penalty':penalty, 'solver':solver, 'warm_start':warm_start}
random_grid_logreg

{'C': [0.1, 0.64, 1.19, 1.73, 2.28, 2.82, 3.37, 3.91, 4.46, 5.0],
 'class_weight': ['balanced', None],
 'fit_intercept': [True, False],
 'multi_class': ['auto', 'ovr', 'multinomial'],
 'penalty': ['l2', None],
 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
 'warm_start': [True, False]}

In [ ]:
logreg_cv = LogisticRegression()
lr_random = RandomizedSearchCV(estimator=logreg_cv, param_distributions=random_grid_logreg, n_iter=100, cv=5, verbose=2)
lr_random.fit(x_train_scaled, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END C=1.73, class_weight=None, fit_intercept=True, multi_class=auto, penalty=None, solver=saga, warm_start=False; total time=   0.0s
[CV] END C=1.73, class_weight=None, fit_intercept=True, multi_class=auto, penalty=None, solver=saga, warm_start=False; total time=   0.0s
[CV] END C=1.73, class_weight=None, fit_intercept=True, multi_class=auto, penalty=None, solver=saga, warm_start=False; total time=   0.0s
[CV] END C=1.73, class_weight=None, fit_intercept=True, multi_class=auto, penalty=None, solver=saga, warm_start=False; total time=   0.0s
[CV] END C=1.73, class_weight=None, fit_intercept=True, multi_class=auto, penalty=None, solver=saga, warm_start=False; total time=   0.0s
[CV] END C=1.73, class_weight=None, fit_intercept=True, multi_class=ovr, penalty=None, solver=sag, warm_start=False; total time=   0.0s
[CV] END C=1.73, class_weight=None, fit_intercept=True, multi_class=ovr, penalty=None, solver=sag, warm_start=F

RandomizedSearchCV(cv=5, estimator=LogisticRegression(), n_iter=100,
                   param_distributions={'C': [0.1, 0.64, 1.19, 1.73, 2.28, 2.82,
                                              3.37, 3.91, 4.46, 5.0],
                                        'class_weight': ['balanced', None],
                                        'fit_intercept': [True, False],
                                        'multi_class': ['auto', 'ovr',
                                                        'multinomial'],
                                        'penalty': ['l2', None],
                                        'solver': ['newton-cg', 'lbfgs',
                                                   'liblinear', 'sag', 'saga'],
                                        'warm_start': [True, False]},
                   verbose=2)

In [ ]:
lr_random.best_params_

{'warm_start': True,
 'solver': 'lbfgs',
 'penalty': 'l2',
 'multi_class': 'auto',
 'fit_intercept': True,
 'class_weight': None,
 'C': 2.82}

In [ ]:
logreg = LogisticRegression(warm_start= True,
 solver= 'lbfgs',
 penalty= 'l2',
 multi_class= 'auto',
 fit_intercept= True,
 class_weight= None,
 C= 2.82)
logreg.fit(x_train_scaled, y_train)

y_pred = logreg.predict(x_test)
print(metrics.accuracy_score(y_test, y_pred))

0.6853146853146853
